In [ ]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error


In [ ]:
df = pd.read_csv("../input/tabularplaygroundfolds/train_folds.csv")
df_test = pd.read_csv("../input/tabular-playground-series-sep-2021/test.csv")
sample_submission = pd.read_csv("../input/tabular-playground-series-sep-2021/sample_solution.csv")

In [ ]:
useful_features = [c for c in df.columns if c not in("id","claim","kfold")]
df_test = df_test[useful_features]


## missing values    
mean_value_df = df[useful_features].mean()
df.fillna(value=mean_value_df,inplace=True)

mean_value_df_test = df_test.mean()
df_test.fillna(value=mean_value_df_test,inplace=True)

In [ ]:
final_predictions=[]
for fold in range(5):
    xtrain = df[df.kfold !=fold].reset_index(drop=True)
    xvalid = df[df.kfold ==fold].reset_index(drop=True)
    xtest = df_test.copy()
    
    ytrain = xtrain.claim
    yvalid = xvalid.claim
    
    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features]
    
    
    model = RandomForestClassifier(n_jobs=4, random_state=fold, verbose=10)
    model.fit(xtrain, ytrain)
    preds_valid = model.predict(xvalid)
    test_preds = model.predict(xtest)
    final_predictions.append(test_preds)
    print(fold, mean_squared_error(yvalid, preds_valid, squared=False))

In [ ]:
preds = np.mean(np.column_stack(final_predictions),axis=1)

In [ ]:
preds.shape

In [ ]:
sample_submission.target=preds
sample_submission.to_csv("submission.csv",index=False)